# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [61]:
# Import libraries
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.corpus import stopwords
import re
import nltk
from nltk.probability import ConditionalFreqDist
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords

In [133]:
import pandas as pd

# Define the desired column names
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']

# Try different encodings
encodings = ['utf-8', 'latin1', 'windows-1252']
for encoding in encodings:
    try:
        # Read the CSV file with specified column names and sample 20000 rows
        tweets = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding=encoding, names=column_names, header=None).sample(10000)
        break  # If successful, exit the loop
    except UnicodeDecodeError:
        print(f"Failed to read with encoding {encoding}. Trying next encoding...")

print("Data loaded successfully.")

Failed to read with encoding utf-8. Trying next encoding...
Data loaded successfully.


In [134]:
tweets

,target,id,date,flag,user,text
1358855,4,2048305207,Fri Jun 05 14:59:27 PDT 2009,NO_QUERY,Dastream,time for food and ice cream gummy
1267721,4,1999911010,Mon Jun 01 21:02:54 PDT 2009,NO_QUERY,ahalady89,At school now. I was able to finish all the wo...
96118,0,1792477802,Wed May 13 23:15:18 PDT 2009,NO_QUERY,CosmetologyGuru,just waxed my brows... doing it yourself kinda...
393941,0,2055517461,Sat Jun 06 09:27:45 PDT 2009,NO_QUERY,evanh86,Just past a baby deer on the road... Dead.
1458333,4,2063659662,Sun Jun 07 03:33:59 PDT 2009,NO_QUERY,sahtaylor,Check out http://www.twitterbackgrounds.com/
...,...,...,...,...,...,...
1212985,4,1989306664,Mon Jun 01 01:03:50 PDT 2009,NO_QUERY,jaepamandanan,Excited for my chicken tenders
279534,0,1991813004,Mon Jun 01 07:42:31 PDT 2009,NO_QUERY,anuptn,Worst road in Chennai is Spur Tank Road......
1120224,4,1973913775,Sat May 30 12:04:45 PDT 2009,NO_QUERY,MeLiSsSsA_,Yuju! Tengo 100 Followers!
398831,0,2056993192,Sat Jun 06 12:10:22 PDT 2009,NO_QUERY,LoreniaCarranza,@greendazzle lost lost lost i miss you...


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [135]:
# your code here
def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
        # Remove numbers
    s = re.sub(r'\d+', ' ', s)
    
    # Remove URLs
    s = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', s)
    
    # Remove special characters
    s = re.sub(r'[^\w\s]', ' ', s)
    
    return s.lower().strip()

def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    return nltk.word_tokenize(s)


def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun if no suitable tag is found

def stem_and_lemmatize(l):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    stemmed_and_lemmatized = []
    for word in l:
        pos_tag = nltk.pos_tag([word])[0][1]  # Get the POS tag for lemmatization
        wordnet_pos = get_wordnet_pos(pos_tag)
        lemmatized_word = lemmatizer.lemmatize(word, pos=wordnet_pos)
        stemmed_and_lemmatized.append(stemmer.stem(lemmatized_word))

    return stemmed_and_lemmatized
def remove_stopwords(l):
    """
    Remove English stopwords from a list of strings.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after stop words are removed.
    """
    stop_words = set(stopwords.words('english'))
    filtered_list = [word for word in l if word.lower() not in stop_words]
    return filtered_list


In [136]:
tweets['text'] = tweets['text'].apply(str)

tweets['text_processed'] = tweets['text'].apply(clean_up).apply(tokenize).apply(stem_and_lemmatize).apply(remove_stopwords)
tweets

,target,id,date,flag,user,text,text_processed
1358855,4,2048305207,Fri Jun 05 14:59:27 PDT 2009,NO_QUERY,Dastream,time for food and ice cream gummy,"[time, food, ice, cream, gummi]"
1267721,4,1999911010,Mon Jun 01 21:02:54 PDT 2009,NO_QUERY,ahalady89,At school now. I was able to finish all the wo...,"[school, abl, finish, work, due, tonight, than..."
96118,0,1792477802,Wed May 13 23:15:18 PDT 2009,NO_QUERY,CosmetologyGuru,just waxed my brows... doing it yourself kinda...,"[wax, brow, kinda, suck, know, pain, happen]"
393941,0,2055517461,Sat Jun 06 09:27:45 PDT 2009,NO_QUERY,evanh86,Just past a baby deer on the road... Dead.,"[past, babi, deer, road, dead]"
1458333,4,2063659662,Sun Jun 07 03:33:59 PDT 2009,NO_QUERY,sahtaylor,Check out http://www.twitterbackgrounds.com/,[check]
...,...,...,...,...,...,...,...
1212985,4,1989306664,Mon Jun 01 01:03:50 PDT 2009,NO_QUERY,jaepamandanan,Excited for my chicken tenders,"[excit, chicken, tender]"
279534,0,1991813004,Mon Jun 01 07:42:31 PDT 2009,NO_QUERY,anuptn,Worst road in Chennai is Spur Tank Road......,"[bad, road, chennai, spur, tank, road]"
1120224,4,1973913775,Sat May 30 12:04:45 PDT 2009,NO_QUERY,MeLiSsSsA_,Yuju! Tengo 100 Followers!,"[yuju, tengo, follow]"
398831,0,2056993192,Sat Jun 06 12:10:22 PDT 2009,NO_QUERY,LoreniaCarranza,@greendazzle lost lost lost i miss you...,"[greendazzl, lose, lose, lose, miss, miss, tal..."


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [137]:
# your code here
tweets_100 = tweets.head(100)
bag_of_words = nltk.FreqDist()

for tweet in tweets_100['text_processed']:
    for element in tweet:
        bag_of_words[element] += 1
        


top_words = bag_of_words.most_common(5000)
top_words_only = [word for word, freq in top_words]

print(top_words_only)

['get', 'day', 'go', 'thank', 'love', 'great', 'x', 'onli', 'whi', 'hope', 'know', 'im', 'amp', 'good', 'u', 'think', 'luv', 'time', 'hey', 'ever', 'realli', 'lol', 'haha', 'well', 'one', 'would', 'take', 'miss', 'see', 'quot', 'work', 'happen', 'readi', 'guy', 'loser', 'follow', 'pleas', 'last', 'tell', 'sick', 'long', 'yay', 'thi', 'yeah', 'want', 'onc', 'bad', 'cant', 'tonight', 'suck', 'pain', 'sure', 'live', 'best', 'song', 'proud', 'still', 'lord', 'fun', 'lose', 'help', 'home', 'drama', 'week', 'cheer', 'probabl', 'like', 'w', 'tweet', 'either', 'movi', 'wait', 'need', 'sleep', 'damn', 'busi', 'feel', 'alon', 'sound', 'right', 'morn', 'stop', 'hour', 'chocol', 'peopl', 'say', 'phone', 'oh', 'night', 'torrent', 'excit', 'new', 'team', 'thing', 'je', 'stupid', 'happi', 'show', 'yoga', 'insid', 'make', 'concert', 'food', 'ice', 'cream', 'gummi', 'school', 'abl', 'finish', 'due', 'lara', 'wax', 'brow', 'kinda', 'past', 'babi', 'deer', 'road', 'dead', 'check', 'gettin', 'train', 'ses

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [138]:
def find_features(document):
    words = set(document)
    features = {}
    for w in top_words:
        features[w] = (w in words)

    return features
feature_sets = [(find_features(tweet), target) for (tweet, target) in list(zip(tweets['text_processed'], tweets['target']))]
print(len(feature_sets))

10000


In [139]:
training_set = feature_sets[:8000] 
testing_set = feature_sets[8000:]
classifier = nltk.NaiveBayesClassifier.train(training_set)
print(nltk.classify.accuracy(classifier, testing_set))
classifier.show_most_informative_features(5) 

0.5015217391304347
Most Informative Features
            ('funni', 1) = False               0 : 4      =      1.0 : 1.0
             ('sooo', 1) = False               0 : 4      =      1.0 : 1.0
             ('norm', 1) = False               0 : 4      =      1.0 : 1.0
     ('jbmaremarejb', 1) = False               0 : 4      =      1.0 : 1.0
               ('bd', 1) = False               0 : 4      =      1.0 : 1.0


### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [140]:
# your code here
print(nltk.classify.accuracy(classifier, testing_set))

0.5015217391304347


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here